In [2]:
import requests
from bs4 import BeautifulSoup
import boto3
import nltk

nltk.data.path = ['C:/Users/theog/nltk_data']

comprehend = boto3.client('comprehend', region_name='us-west-2')

URL = "https://twstalker.com/Bell" # pour l'entreprise qui parle d'elle-même
URL = "https://twstalker.com/search/bell-canada" # pour les clients qui en parlent
page = requests.get(URL)

if page.status_code == 200:
    print("Page récupérée avec succès.")
else:
    print(f"Erreur lors de la récupération de la page. Code d'erreur : {page.status_code}")

soup = BeautifulSoup(page.content, "html.parser")

tweets = []
job_elements = soup.find_all("p")

nb_tweets = 30

for i, job_element in enumerate(job_elements[:nb_tweets+1]):
    element = job_element.text
    tweets.append(element)

tweets.remove(tweets[0])
print(f"Nombre de tweets extraits : {len(tweets)}")

positive_tweets = 0
negative_tweets = 0
neutral_tweets = 0
mixed_tweets = 0

sample_positive = None
sample_negative = None
sample_neutral = None
sample_mixed = None

def classify_sentiment_comprehend(tweet):
    response = comprehend.detect_sentiment(Text=tweet, LanguageCode='en')
    sentiment = response['Sentiment']
    return sentiment

for tweet in tweets:
    sentiment = classify_sentiment_comprehend(tweet)
    if sentiment == 'POSITIVE':
        positive_tweets += 1
        if sample_positive is None:
            sample_positive = tweet
    elif sentiment == 'NEGATIVE':
        negative_tweets += 1
        if sample_negative is None:
            sample_negative = tweet
    elif sentiment == 'NEUTRAL':
        neutral_tweets += 1
        if sample_neutral is None:
            sample_neutral = tweet
    elif sentiment == 'MIXED':
        mixed_tweets += 1
        if sample_mixed is None:
            sample_mixed = tweet

total_tweets = len(tweets)
if total_tweets > 0:
    positive_ratio = positive_tweets / total_tweets
    negative_ratio = negative_tweets / total_tweets
    neutral_ratio = neutral_tweets / total_tweets
    mixed_ratio = mixed_tweets / total_tweets
    
    print(f"\nRatio de tweets positifs : {positive_ratio:.2%}")
    print(f"Ratio de tweets négatifs : {negative_ratio:.2%}")
    print(f"Ratio de tweets neutres : {neutral_ratio:.2%}")
    print(f"Ratio de tweets mixtes : {mixed_ratio:.2%}")
else:
    print("Aucun tweet à analyser.")

print(f"\nNombre de tweets positifs : {positive_tweets}")
print(f"Nombre de tweets négatifs : {negative_tweets}")
print(f"Nombre de tweets neutres : {neutral_tweets}")
print(f"Nombre de tweets mixtes : {mixed_tweets}")

print("\nExemple de tweet positif :", sample_positive)
print("Exemple de tweet négatif :", sample_negative)
print("Exemple de tweet neutre :", sample_neutral)
print("Exemple de tweet mixte :", sample_mixed)


Page récupérée avec succès.
Nombre de tweets extraits : 16

Ratio de tweets positifs : 12.50%
Ratio de tweets négatifs : 18.75%
Ratio de tweets neutres : 62.50%
Ratio de tweets mixtes : 6.25%

Nombre de tweets positifs : 2
Nombre de tweets négatifs : 3
Nombre de tweets neutres : 10
Nombre de tweets mixtes : 1

Exemple de tweet positif : When I sold my Bell Canada stock, I moved the proceeds to a U.S. Quality ETF that had a favourable valuation, and a wonderful volatility history. Take a look ... cutthecrapinvesting.com/2024/10/20/ish… 
Exemple de tweet négatif : @VirginPlus I am not impressed. Your modem dies on me and “support” tells me I have to bring your equipment to a #bell #canada store.

How much am I paying you so I can do your job?

#fail #virgin #badservice 
Exemple de tweet neutre : @SamiParvezQadir @Bell Hi Sami, thank you for your inquiry. That does not appear to be coming from Bell Canada. If you have any other questions, please send us a DM. ^JL 
Exemple de tweet mixte :